In [148]:
import pandas as pd
import json
from pandas.io.json import json_normalize
from collections import defaultdict

In [149]:
with open('/Users/mustafacanan/Desktop/DataFolder/data/world_bank_projects.json','r') as json_file:
    worldbank_data = json.load(json_file)
print(type(worldbank_data))
print(type(worldbank_data[0]))

<class 'list'>
<class 'dict'>


In [150]:
#define json string
for key,value in worldbank_data[0].items():
    print(key + ':',value)

sector: [{'Name': 'Primary education'}, {'Name': 'Secondary education'}, {'Name': 'Public administration- Other social services'}, {'Name': 'Tertiary education'}]
supplementprojectflg: N
projectfinancialtype: IDA
prodline: PE
mjtheme: ['Human development']
idacommamt: 130000000
impagency: MINISTRY OF EDUCATION
project_name: Ethiopia General Education Quality Improvement Project II
mjthemecode: 8,11
closingdate: 2018-07-07T00:00:00Z
totalcommamt: 130000000
id: P129828
mjsector_namecode: [{'code': 'EX', 'name': 'Education'}, {'code': 'EX', 'name': 'Education'}, {'code': 'BX', 'name': 'Public Administration, Law, and Justice'}, {'code': 'EX', 'name': 'Education'}]
docty: Project Information Document,Indigenous Peoples Plan,Project Information Document
sector1: {'Percent': 46, 'Name': 'Primary education'}
lendinginstr: Investment Project Financing
countrycode: ET
sector2: {'Percent': 26, 'Name': 'Secondary education'}
totalamt: 130000000
mjtheme_namecode: [{'code': '8', 'name': 'Human deve

In [151]:
#use normalization to create tables from nested element
worldbank_normalize=json_normalize(worldbank_data)
print(worldbank_normalize.head())

                   _id.$oid approvalfy board_approval_month  \
0  52b213b38594d8a2be17c780       1999             November   
1  52b213b38594d8a2be17c781       2015             November   
2  52b213b38594d8a2be17c782       2014             November   
3  52b213b38594d8a2be17c783       2014              October   
4  52b213b38594d8a2be17c784       2014              October   

      boardapprovaldate                                 borrower  \
0  2013-11-12T00:00:00Z  FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA   
1  2013-11-04T00:00:00Z                    GOVERNMENT OF TUNISIA   
2  2013-11-01T00:00:00Z   MINISTRY OF FINANCE AND ECONOMIC DEVEL   
3  2013-10-31T00:00:00Z   MIN. OF PLANNING AND INT'L COOPERATION   
4  2013-10-31T00:00:00Z                      MINISTRY OF FINANCE   

            closingdate                              country_namecode  \
0  2018-07-07T00:00:00Z  Federal Democratic Republic of Ethiopia!$!ET   
1                   NaN                      Republic of Tunisia!$

In [152]:
#load as Pandas dataframe
worldbank_df=pd.read_json('/Users/mustafacanan/Desktop/DataFolder/data/world_bank_projects.json',orient='records')
print(type(worldbank_df))
print(worldbank_df.theme_namecode.head())

<class 'pandas.core.frame.DataFrame'>
0        [{'code': '65', 'name': 'Education for all'}]
1    [{'code': '24', 'name': 'Other economic manage...
2    [{'code': '47', 'name': 'Regional integration'...
3    [{'code': '57', 'name': 'Participation and civ...
4    [{'code': '45', 'name': 'Export development an...
Name: theme_namecode, dtype: object


In [153]:
#Find the 10 countries with most projects
print('10 Countries With The Most Projects')
print(worldbank_normalize['countryname'].value_counts().head(10))

10 Countries With The Most Projects
Republic of Indonesia              19
People's Republic of China         19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Kingdom of Morocco                 12
Nepal                              12
People's Republic of Bangladesh    12
Africa                             11
Republic of Mozambique             11
Name: countryname, dtype: int64


In [183]:
#Find the top 10 major project themes
print('Top 10 Major Project Themes')
worldbank_normalize2=json_normalize(worldbank_data,'mjtheme_namecode')
print(worldbank_normalize2['name'].value_counts().head(10))

Top 10 Major Project Themes
Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64


In [193]:
#checking the data info
print(worldbank_normalize2.describe())

        code                                          name
count   1499                                          1499
unique    11                                            12
top       11  Environment and natural resources management
freq     250                                           223


In [203]:
#theme names and codes map
worldbank_normalize2.drop_duplicates()
worldbank_clean={}
for row in worldbank_normalize2.itertuples():
    if row[2] != '':
        worldbank_clean[row[1]] = row[2]
print(worldbank_clean)
        

{'8': 'Human development', '1': 'Economic management', '6': 'Social protection and risk management', '5': 'Trade and integration', '2': 'Public sector governance', '11': 'Environment and natural resources management', '7': 'Social dev/gender/inclusion', '4': 'Financial and private sector development', '10': 'Rural development', '9': 'Urban development', '3': 'Rule of law'}


In [207]:
#Filling the missing values
for row in worldbank_normalize2.itertuples():
    if row[2] == '':
        worldbank_normalize2.at(row[0], 'name', worldbank_clean[row[1]])
print(worldbank_normalize2.name.head(10))

0                               Human development
1    Environment and natural resources management
2                             Economic management
3           Social protection and risk management
4                           Trade and integration
5                        Public sector governance
6    Environment and natural resources management
7           Social protection and risk management
8                     Social dev/gender/inclusion
9                     Social dev/gender/inclusion
Name: name, dtype: object


In [209]:
#Last version of the top 10 major project themes
print(worldbank_normalize2['name'].value_counts().head(10))

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64
